<a href="https://colab.research.google.com/github/Inha-AI/DACON-semiconductor-competition/blob/feature%2FYoonSungLee/find_optimal_units.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=bbba662eac651d5a31fad677e8834a48514797055de445092394c7c98eb874f7
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# 데이터 자료형을 적절히 변형시켜 데이터의 크기를 줄이는 방법

# for col in df_train.columns:
#     col_type = df_train[col].dtypes
#     min1 = df_train[col].min()
#     max1 = df_train[col].max()
#     if str(col_type)[:3] == 'int':
#         df_train[col] = df_train[col].astype(np.int16)
#     else:
#         if min1 > np.finfo(np.float16).min and max1 < np.finfo(np.float16).max:
#             df_train[col] = trdf_trainain[col].astype(np.float16)
#         elif min1 > np.finfo(np.float32).min and max1 < np.finfo(np.float32).max:
#             df_train[col] = df_train[col].astype(np.float32)
#         else:
#             df_train[col] = df_train[col].astype(np.float64)

In [0]:
df_train = pd.read_csv('/gdrive/My Drive/DACON-semiconductor-competition/dataset/train.csv')
df_test = pd.read_csv('/gdrive/My Drive/DACON-semiconductor-competition/dataset/test.csv')

In [0]:
# 독립변수와 종속변수를 분리합니다.

train_X = df_train.iloc[:,4:]
train_Y = df_train.iloc[:,0:4]
test_X = df_test.iloc[:,1:]

In [0]:
# train set을 shuffle하여 다시 train set과 validation set으로 분리합니다.

train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size=0.25)

# Test

In [0]:
# 케라스를 통해 모델 생성을 시작합니다.

def create_model(units):
    model = Sequential()
    model.add(Dense(units=units, input_dim=226, kernel_initializer='he_normal'))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(Dense(units=units, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dense(units=4, activation='linear'))

    return model


# Activation Function 정의

def swish(x) :
    return x * keras.activations.sigmoid(x)


# layer의 특정 units 하에서 학습을 수행한 후, 검증 성능을 출력하는 목적 함수 정의

def train_and_validate(units):
    model = create_model(int(units))
    adam = keras.optimizers.Adam(0.001)
    model.compile(loss='mae', optimizer=adam, metrics=['accuracy'])
    hist = model.fit(train_X, train_Y, epochs=20, batch_size=630,
                    validation_data=(val_X, val_Y))
    best_val_score = max(hist.history['val_acc'])

    return best_val_score

In [0]:
# BayesianOptimization 객체 생성, 실행 및 최종 결과 출력
# 최적의 layer units 수를 찾기 위함

bayes_optimizer = BayesianOptimization(
    f=train_and_validate,
    pbounds={
         'units' : (100, 800)
    },
    random_state=0,
    verbose=1
)

bayes_optimizer.maximize(init_points=3, n_iter=27, acq='ei',xi=0.01)

for i, res in enumerate(bayes_optimizer.res):
    print('iteration {}: \n\t{}'.format(i, res))

|   iter    |  target   |   units   |
-------------------------------------









Train on 607500 samples, validate on 202500 samples
Epoch 1/20





607500/607500 [==============================] - 29s 47us/step - loss: 72.3486 - acc: 0.3736 - val_loss: 67.9759 - val_acc: 0.3303
Epoch 2/20
607500/607500 [==============================] - 18s 29us/step - loss: 27.0557 - acc: 0.7234 - val_loss: 25.4235 - val_acc: 0.7373
Epoch 3/20
607500/607500 [==============================] - 18s 29us/step - loss: 16.6041 - acc: 0.8296 - val_loss: 15.8608 - val_acc: 0.8363
Epoch 4/20
607500/607500 [==============================] - 18s 30us/step - loss: 12.8150 - acc: 0.8668 - val_loss: 14.3804 - val_acc: 0.8540
Epoch 5/20
607500/607500 [==============================] - 17s 29us/step - loss: 10.6862 - acc: 0.8875 - val_loss: 11.5420 - val_acc: 0.8775
Epoch 6/20
607500/607500 [==============================] - 18s 29us/step - loss: 9.3785 - acc: 0.9006 - val_loss: 8.9325 - val_acc: 0.9104
Epoch 7/

In [0]:
print('Final result: ', bayes_optimizer.max)

Final result:  {'target': 0.9581975173950196, 'params': {'units': 799.999308421973}}


### Bayesian Optimization
http://research.sualab.com/introduction/practice/2019/02/19/bayesian-optimization-overview-1.html<br>
http://research.sualab.com/introduction/practice/2019/04/01/bayesian-optimization-overview-2.html<br>

### Swish Activation
https://www.machinecurve.com/index.php/2019/05/30/why-swish-could-perform-better-than-relu/#todays-activation-functions